In [8]:
# Loads gaze-related metrics for each trial and participant and performs statistical analysis for the whole study 

In [9]:
import re
import matplotlib.pyplot as plt
from enum import Enum
import os
import statistics
import numpy as np
import get_tcm as tcm
import csv
import pandas as pd

In [10]:
data_dir="Z:/builds/aplause-mr/UNITY_APP/aplause-mr-20240707-studystate-v1/gazeAnalysis"

file_path = data_dir + "/gaze_classification_analysis_results.csv"

df = pd.read_csv(file_path)
df

,participant_id,trial,group,condition,audio_condition,avatar_condition,gaze_local_pc,gaze_remote_adjacent_pc,gaze_remote_opposite_pc,gaze_others_pc,...,glance_remote_opposite_per_min,mutual_gaze_local_pc,mutual_gaze_local_mean_duration,mutual_gaze_local_per_min,mutual_gaze_remote_opp_pc,mutual_gaze_remote_opp_mean_duration,mutual_gaze_remote_opp_per_min,mutual_gaze_remote_adj_pc,mutual_gaze_remote_adj_mean_duration,mutual_gaze_remote_adj_per_min
0,0,0,0,DIOTIC_VOLUMETRIC,DIOTIC,VOLUMETRIC,18.387516,12.327698,55.786736,86.501951,...,12.795839,2.600780,1.111111,1.404421,37.295189,1.541935,14.512354,4.005202,0.855556,2.808843
1,1,0,0,DIOTIC_VOLUMETRIC,DIOTIC,VOLUMETRIC,5.331599,22.288687,39.817945,67.438231,...,11.235371,2.600780,1.111111,1.404421,12.145644,0.676812,10.767230,2.782835,0.972727,1.716515
2,2,0,0,DIOTIC_VOLUMETRIC,DIOTIC,VOLUMETRIC,3.172952,14.668401,58.179454,76.020806,...,10.299090,1.014304,0.780000,0.780234,37.295189,1.541935,14.512354,2.782835,0.972727,1.716515
3,3,0,0,DIOTIC_VOLUMETRIC,DIOTIC,VOLUMETRIC,13.888166,30.195059,33.211964,77.295189,...,15.136541,1.014304,0.780000,0.780234,12.145644,0.676812,10.767230,4.005202,0.855556,2.808843
4,0,1,0,DIOTIC_ABSTRACT,DIOTIC,ABSTRACT,17.973062,5.012144,66.173548,89.158755,...,9.406050,0.419519,0.475000,0.529918,40.759550,1.619298,15.102672,1.368956,0.885714,0.927357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,3,2,19,SPATIAL_VOLUMETRIC,SPATIAL,VOLUMETRIC,8.477041,19.925203,40.556825,68.959069,...,17.826719,0.000000,-1.000000,0.000000,21.961355,0.794737,16.580096,2.888012,0.695000,2.493247
252,0,3,19,DIOTIC_ABSTRACT,DIOTIC,ABSTRACT,28.726356,6.027743,22.723834,57.477932,...,8.776797,7.717528,0.927273,4.993695,4.590164,0.791304,3.480454,1.488020,0.536364,1.664565
253,1,3,19,DIOTIC_ABSTRACT,DIOTIC,ABSTRACT,24.262295,3.051702,46.506936,73.820933,...,18.612863,7.717528,0.927273,4.993695,16.973518,0.701042,14.527112,0.151324,0.600000,0.151324
254,2,3,19,DIOTIC_ABSTRACT,DIOTIC,ABSTRACT,2.118537,5.346784,18.486759,25.952081,...,11.803279,0.075662,0.150000,0.302648,4.590164,0.791304,3.480454,0.151324,0.600000,0.151324
